In [1]:
import numpy as np
import open3d
import pickle as pkl
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data_dict_loc = '1607-143021244/data_dict.pkl'
pred_dict_loc = '1607-143021244//pred_dict.pkl'

In [3]:
with open(data_dict_loc, 'rb') as file:
    data_dict = pkl.load(file)

with open(pred_dict_loc, 'rb') as file:
    pred_dict = pkl.load(file)

In [4]:
data_dict['points']

array([[  0.       ,  27.5542   , -31.9538   ,  15.9793   ,   0.       ],
       [  0.       ,  27.3868   , -32.1565   ,  15.9963   ,   0.       ],
       [  0.       ,  26.9993   , -32.0985   ,  15.8847   ,   0.       ],
       ...,
       [  0.       ,   1.99034  ,   0.060584 ,  -0.780597 ,   0.       ],
       [  0.       ,   2.01856  ,   0.0490654,  -0.791626 ,   0.       ],
       [  0.       ,   2.06439  ,   0.0375345,  -0.809641 ,   0.       ]],
      dtype=float32)

In [5]:
pred_dict

{'pred_boxes': array([[  1.3119578 ,   5.853373  ,  -0.59230673,   0.41738972,
           0.45552054,   1.5290838 ,   3.477679  ],
        [ 29.494904  ,  16.33713   ,  -0.85819024,   3.4525747 ,
           1.5573701 ,   1.4580625 ,   5.3251176 ],
        [ 24.76013   ,  -9.071726  ,  -0.20101176,   3.4634647 ,
           1.652355  ,   1.6754034 ,   3.3416688 ],
        [ 14.376092  ,   7.541929  ,  -0.48910043,   3.622657  ,
           1.5746876 ,   1.4420577 ,   1.3164713 ],
        [ 21.638906  ,   3.0925167 ,  -0.6536154 ,   3.7388947 ,
           1.6871119 ,   1.3593096 ,   2.5330553 ],
        [  7.889608  , -14.110378  ,  -1.099024  ,   3.9845731 ,
           1.4794515 ,   1.3429484 ,   2.0542426 ],
        [  0.69889635,  21.93235   ,  -0.6637959 ,   3.6904538 ,
           1.507566  ,   1.3925595 ,   5.8991456 ],
        [  8.675017  ,  22.207909  ,  -0.56216985,   3.7920983 ,
           1.5595655 ,   1.3701258 ,   3.5307953 ],
        [ 11.629646  ,  28.37143   ,  -0.46542788,

In [6]:
box_colormap = [
    [1, 1, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 1, 0],
]

In [7]:
def translate_boxes_to_open3d_instance(gt_boxes):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])
    rot = open3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = open3d.geometry.OrientedBoundingBox(center, rot, lwh)

    line_set = open3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    # import ipdb; ipdb.set_trace(context=20)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = open3d.utility.Vector2iVector(lines)

    return line_set, box3d

In [8]:
def draw_box(vis, gt_boxes, color=(0, 1, 0), ref_labels=None, score=None):
    for i in range(gt_boxes.shape[0]):
        line_set, box3d = translate_boxes_to_open3d_instance(gt_boxes[i])
        if ref_labels is None:
            line_set.paint_uniform_color(color)
        else:
            line_set.paint_uniform_color(box_colormap[ref_labels[i]])

        vis.add_geometry(line_set)

        # if score is not None:
        #     corners = box3d.get_box_points()
        #     vis.add_3d_label(corners[5], '%.2f' % score[i])
    return vis


In [9]:
def draw_scenes(points, gt_boxes=None, ref_boxes=None, ref_labels=None, ref_scores=None, point_colors=None, draw_origin=True):

    vis = open3d.visualization.Visualizer()
    vis.create_window()

    vis.get_render_option().point_size = 1.0
    vis.get_render_option().background_color = np.zeros(3)

    # draw origin
    if draw_origin:
        axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
        vis.add_geometry(axis_pcd)

    pts = open3d.geometry.PointCloud()
    pts.points = open3d.utility.Vector3dVector(points[:, :3])

    vis.add_geometry(pts)
    if point_colors is None:
        pts.colors = open3d.utility.Vector3dVector(np.ones((points.shape[0], 3)))
    else:
        pts.colors = open3d.utility.Vector3dVector(point_colors)

    if gt_boxes is not None:
        vis = draw_box(vis, gt_boxes, (0, 0, 1))

    if ref_boxes is not None:
        vis = draw_box(vis, ref_boxes, (0, 1, 0), ref_labels, ref_scores)

    vis.run()
    vis.destroy_window()

In [10]:
draw_scenes(data_dict['points'][:, 1:], ref_boxes=pred_dict['pred_boxes'], ref_scores=pred_dict['pred_scores'], ref_labels=pred_dict['pred_labels'])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
